In [1]:
import numpy as np
from pyscf import gto, scf, fci


$$
\begin{split}
E_H[\gamma_1] & =  \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{a,\kappa} \tilde{\tilde{c}}_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H[\gamma_1] & =  \iint \sum_a^M \sum_b^M n_a n_b \frac{\eta_a(r)^{\ast} \eta_a(r) \eta_b(r')^{\ast}  \eta_b(r')}{|r-r'|} dr dr' \\
E_H[\gamma_1] & =  \iint \frac{\gamma_1(r,r) \gamma_1(r',r')}{|r-r'|} dr dr' \\
\end{split}
$$

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [2]:
def ONERDMFT_hartree_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu,nu,kappa,lamda]

    return 2*energy



$$
\begin{split}
E_x[\gamma_1] & = -\frac{1}{2} \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{b,\kappa} \tilde{\tilde{c}}_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x[\gamma_1] & = -\frac{1}{2} \iint \sum_a^M \sum_b^M n_a n_b \frac{\eta_a(r)^{\ast} \eta_b(r) \eta_b(r')^{\ast}  \eta_a(r')}{|r-r'|} dr dr' \\
E_x[\gamma_1] & = -\frac{1}{2} \iint \frac{\gamma_1(r,r') \gamma_1(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [3]:
def ONERDMFT_exchange_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,b]*C[lamda,a]*Fouridx[mu,nu,kappa,lamda]

    return -1.*energy


$$
\begin{split}
E_{xc}[\gamma_1] & = -\frac{1}{2} \sum_a^M \sum_b^M \sqrt{n_a n_b} \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{b,\kappa} \tilde{\tilde{c}}_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_{xc}[\gamma_1] & = -\frac{1}{2} \iint \sum_a^M \sum_b^M \sqrt{n_a n_b} \frac{\eta_a(r)^{\ast} \eta_b(r) \eta_b(r')^{\ast}  \eta_a(r')}{|r-r'|} dr dr' \\
E_{xc}[\gamma_1] & = -\frac{1}{2} \iint \frac{\gamma_1^{\frac{1}{2}}(r,r') \gamma_1^{\frac{1}{2}}(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [4]:
def ONERDMFT_Mueller_functional(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return -1.*energy

In [5]:
def ONERDMFT_Mueller_Full(Fouridx, C, n, mol, h, E_h):
    U = ONERDMFT_hartree_energy(Fouridx, C, n, mol)
    E_x = ONERDMFT_exchange_energy(Fouridx, C, n, mol)
    E_xc = ONERDMFT_Mueller_functional(Fouridx, C, n, mol)
    E_c = E_xc - E_x
    print(E_h, U, E_xc, E_x, E_c)
    return E_h + U + E_xc

In [6]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')


## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()

N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)
gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)

occu, naturalC = np.linalg.eigh(gamma)

#print(gamma)
#print(np.matmul(np.matmul(naturalC.T,np.diag(occu)), naturalC))

converged SCF energy = -2.85516042615445


In [7]:
for i, n  in enumerate(occu):
    if n < 0:
        occu[i] = 0 

In [8]:
# run FCI 
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)

In [9]:
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIgamma = FCIgamma/2.0
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)

#occu = np.diagonal(np.matmul(np.diag(occu), S))

#print(FCIoccu, occu)

print(FCIoccu,  sum(FCIoccu))

[0.00432351 0.99567649] 0.9999999999999999


In [10]:
naturalCTT = np.matmul(C,naturalC)
FCInaturalCTT = np.matmul(C,naturalC)

print(np.trace(np.matmul(h,gamma))+ONERDMFT_hartree_energy(eri, naturalCTT, occu, mol)+ONERDMFT_exchange_energy(eri, naturalCTT, occu, mol))
print(np.trace(np.matmul(h,FCIgamma))+ONERDMFT_hartree_energy(eri, FCInaturalCTT, FCIoccu, mol)+ONERDMFT_exchange_energy(eri, FCInaturalCTT, FCIoccu, mol))

-0.8527190734922447
-0.8522161940245407


In [11]:
print(f"E FCI: {e:2.4f}; E HF: {mf.e_tot:2.4f}; E Mu:{ONERDMFT_Mueller_Full(eri, FCInaturalCTT, FCIoccu, mol, h, 2*np.trace(np.matmul(h,P))):2.4f} E_c {e-mf.e_tot:2.4f}")

-3.882067595029823 2.0508984339955805 -1.0556561344958408 -1.0200211602942235 -0.03563497420161732
E FCI: -2.8702; E HF: -2.8552; E Mu:-2.8868 E_c -0.0150


In [12]:
print("Mueller Energy, Exchange Energy of HF-gamma:", ONERDMFT_Mueller_functional(eri, naturalCTT, occu, mol),ONERDMFT_exchange_energy(eri, naturalCTT, occu, mol))
print("Mueller Energy, Exchange Energy of FCI-gamma:", ONERDMFT_Mueller_functional(eri, FCInaturalCTT, FCIoccu, mol),ONERDMFT_exchange_energy(eri, FCInaturalCTT, FCIoccu, mol))
print("Mueller Correlation Energy:", ONERDMFT_Mueller_functional(eri, FCInaturalCTT, FCIoccu, mol)-ONERDMFT_exchange_energy(eri, FCInaturalCTT, FCIoccu, mol))


Mueller Energy, Exchange Energy of HF-gamma: -1.0269071688753768 -1.0269071688753773
Mueller Energy, Exchange Energy of FCI-gamma: -1.0556561344958408 -1.0200211602942235
Mueller Correlation Energy: -0.03563497420161732


In [13]:
#E_h = np.trace(np.matmul(h, np.matmul(np.matmul(C.T,np.diagonal(n)), C)))

print(f"Hartree Fock Eotal Energy {mf.e_tot}")
print(f"Mueller Total Energy from FCI gamma {ONERDMFT_Mueller_Full(eri, FCInaturalCTT, FCIoccu, mol, h, 2*np.trace(np.matmul(h,P)))}")
print(f"Mueller Total Energy from HF gamma {ONERDMFT_Mueller_Full(eri, naturalCTT, occu, mol, h, 2*np.trace(np.matmul(h,P)))}")


Hartree Fock Eotal Energy -2.8551604261544465
-3.882067595029823 2.0508984339955805 -1.0556561344958408 -1.0200211602942235 -0.03563497420161732
Mueller Total Energy from FCI gamma -2.886825295530083
-3.882067595029823 2.0538143377507545 -1.0269071688753768 -1.0269071688753773 4.440892098500626e-16
Mueller Total Energy from HF gamma -2.855160426154445
